# Background
In order to find the CCE strains
We need the spherical harmonics coefficients of the adm variables on a nested set of spherical shells at different time step. These are written to a binary format (.bin). The radius of the nested shell are set to be the Chebyshev collocation points, which we then use to interpolate to a set radius in between the inner and outer most radii and calculate the derivatives. These data serve as the boundary condition of `Spectre CCE`.

# Requirements

1. [Spectre CCE executable](https://spectre-code.org/tutorial_cce.html)

2. [BMS trasnformation package (scri)](https://scri.readthedocs.io/en/latest/README.html)

3. python packages: `numpy`, `h5py`, `scipy`, `matplotlib`


# Compute CCE strains using `Spectre CCE`

1. We first convert the boundary condition data, saved in binray format, into a format readable by the `Spectre CCE`. This is done by the `./athk_to_spectre.py` python script. Before lunching the script, we need to know the `dump_extraction_radius`, which can be find in the `AthenaK` parameter file. We usually set the `dump_extraction_radius` parameter at the middle of inner and outer radii of the shell where the metric data are interplated.

```bash

# converting binary dump data into spectre reable format:
./athk_to_spectre.py -ftype bin -radius [dump_extraction_radius] -d_out [output_directory]

```

2. Having run the convertor script, we see inside the output directory there is a `h5` file readable by `Spectre CCE`. The file name is similar to `CceRXXXX.h5`, where `XXXX` specifies the extraction radius. We specify the path of this file inside the `Spectre CCE` yaml parameter file `CharacteristicExtract.yaml`; namely, we set the paramter `BoundaryDataFilename = path/to/CceRXXXX.h5` inside `CharacteristicExtract.yaml`. Additionally, we set the parameter `H5IsBondiData = False`, as the boundary data are not in Bondi format, see more about Bondi format [here](https://spectre-code.org/tutorial_cce.html#autotoc_md58).
We lunch the `Spectre CCE` as follows:

```bash
./CharacteristicExtract --input-file CharacteristicExtract.yaml

```


# BMS transformation

Since the output of `Spectre CCE` is in different that super rest frame, we need to apply a BMS transformation on the GW waves, see [Frame fixing](https://spectre-code.org/tutorial_cce.html#autotoc_md65) and [Scri](https://scri.readthedocs.io/en/latest/tutorial_abd.html#loading-cce-data-and-adjusting-the-bms-frame).
The following script shows how in practice apply a BMS transformation:

```python

import scri
import matplotlib.pyplot as plt
import numpy as np

# transform to the superrest frame
w = scri.WaveformModes()
abd = scri.create_abd_from_h5(
      file_name="/path/to/CharacteristicExtractReduction.h5",
      file_format="spectrecce_v1",
      ch_mass=1.0,
      t_0_superrest=t0, # some time after junk radiation
      padding_time=w0 # about two orbital periods
    )
h = abd.h

# the wavefroms is transformed, now plot h_22, where h.t the the time
plt.plot(h.t, h.data[:, h.index(2,2)])

# you can access to real and imaginary part of h as:
re = h.data[:,h.index(2,2)].real
im = h.data[:,h.index(2,2)].imag
```

# Debugging tools

The `debug_athk_to_spectre.py` script provides tools to analyse and hence debug the transform output of `athk_to_spectre.py` script. One can look at a particular mode at different time and hence check for the convergence of sherical harmonic modes or radial or time derivatives of each boundary condition quantities. The following example exhibits the real $(2,2)$ mode of $g_{xx}(t)$ metric in spherical harmonic:

```bash

./debug_athk_to_spectre.py -debug plot_simple -dout [output_directory] \
  -fpath [/path/to/CceRXXXX.h5]  -field_name "gxx" -field_mode "Re(2,2)"

```



# Background

To compute the _CCE strains_, we need the **spherical harmonics coefficients** of the ADM variables on a nested set of spherical shells at different time steps. These coefficients are written to a binary format (`.bin`).

The radii of these nested shells are set using **Chebyshev collocation points**, which are used to interpolate the data to a set radius between the innermost and outermost shells. Derivatives are also computed at this radius. These data serve as the **boundary conditions** for `Spectre CCE`.

---

# Requirements

1. [`Spectre CCE` executable](https://spectre-code.org/tutorial_cce.html)  
2. [`scri` (BMS transformation package)](https://scri.readthedocs.io/en/latest/README.html)  
3. Python packages:  
   - `numpy`  
   - `h5py`  
   - `scipy`  
   - `matplotlib`  

---

# Compute CCE Strains using `Spectre CCE`

### Step 1: Convert Boundary Data to Spectre Format

First, convert the binary boundary condition data into a format readable by `Spectre CCE` using the script `athk_to_spectre.py`.

Before launching the script, identify the `dump_extraction_radius`, which can be found in the AthenaK parameter file. This radius is typically set midway between the inner and outer radii of the shell where metric data is interpolated.

```bash
# Convert binary dump data to Spectre-readable format:
./athk_to_spectre.py -ftype bin -radius [dump_extraction_radius] -d_out [output_directory]
```

### Step 2: Run `Spectre CCE`

After conversion, the output directory will contain an `.h5` file, named like `CceRXXXX.h5`, where `XXXX` denotes the extraction radius.

Set this file path in the `CharacteristicExtract.yaml` config file:

```yaml
BoundaryDataFilename: path/to/CceRXXXX.h5
H5IsBondiData: false  # Since the data is not in Bondi format
```

Run `Spectre CCE` with:

```bash
./CharacteristicExtract --input-file CharacteristicExtract.yaml
```

For more on the Bondi format, refer to the [Spectre CCE tutorial](https://spectre-code.org/tutorial_cce.html#autotoc_md58).

---

# BMS Transformation

The output from `Spectre CCE` is not in the super rest frame. To transform the waveform, apply a **BMS transformation** using `scri`. See:

- [Frame fixing](https://spectre-code.org/tutorial_cce.html#autotoc_md65)
- [Scri tutorial](https://scri.readthedocs.io/en/latest/tutorial_abd.html#loading-cce-data-and-adjusting-the-bms-frame)

Example:

```python
import scri
import matplotlib.pyplot as plt
import numpy as np

# Load and transform waveform to super rest frame
w = scri.WaveformModes()
abd = scri.create_abd_from_h5(
    file_name="/path/to/CharacteristicExtractReduction.h5",
    file_format="spectrecce_v1",
    ch_mass=1.0,
    t_0_superrest=t0,  # time after junk radiation
    padding_time=w0    # ~2 orbital periods
)
h = abd.h

# Plot h_22 mode
plt.plot(h.t, h.data[:, h.index(2, 2)])

# Access real and imaginary parts
re = h.data[:, h.index(2, 2)].real
im = h.data[:, h.index(2, 2)].imag
```

---

# Debugging Tools

Use the `debug_athk_to_spectre.py` script to analyze and debug the output of `athk_to_spectre.py`. You can inspect specific spherical harmonic modes over time, or examine convergence and derivatives.

Example: Plot the real part of the \((2,2)\) mode of \(g_{xx}(t)\):

```bash
./debug_athk_to_spectre.py -debug plot_simple \
  -dout [output_directory] \
  -fpath [/path/to/CceRXXXX.h5] \
  -field_name "gxx" -field_mode "Re(2,2)"
```
